In [2]:
from hmmlearn import hmm
from abc_utils import *
import numpy as np
from tqdm import tqdm
RANDOM_STATE = 42

In [3]:
train_set, _ = load_harmonization_train_test()

In [20]:
abc_texts = train_set.sample(2, random_state=RANDOM_STATE)['output']

In [21]:
# list_o_dfs = [abc_to_dataframe(abc_text) for abc_text in tqdm(abc_texts)]

In [22]:
l_o_df = []
l_o_df_lengths = []
for abc_text in tqdm(abc_texts):
    try:
        df = abc_to_dataframe(abc_text)
    except Exception as e:
        print("bugged out", e)
    lengths = len(df)
    l_o_df.append(df)
    l_o_df_lengths.append(lengths)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  4.10it/s]


In [23]:
music_df = pd.concat(l_o_df)
state_chords = music_df.chord.values
obs_melody = music_df.melody.values
symbols = sorted(list(set(obs_melody)))
print(obs_melody.shape)
print(np.unique(state_chords))

(236,)
[ 0  1  5  6  8 10 15 22]


In [24]:
h = hmm.CategoricalHMM(n_components=len(np.unique(state_chords)), n_iter=1000, tol=1e-6)
h.fit(obs_melody.reshape(-1, 1), lengths=l_o_df_lengths)
pi = h.startprob_
A = h.transmat_.T
B = h.emissionprob_.T

Fitting a model with 775 free scalar parameters with only 236 data points will result in a degenerate solution.


In [27]:
# try to predict the states from an observation
test_melody = obs_melody[:l_o_df_lengths[0]]
test_chords = state_chords[:l_o_df_lengths[0]]

# do the prediction
log_prob, pred = h.decode(test_melody.reshape(-1, 1))
print("Pred", np.unique(state_chords)[pred])
print("Actu", test_chords)

Pred [ 5  5  5  1  6 15 10  0 22  8 22  8  0 22  8  6 15  6 15  6 15  6 15 10
  0 22  8 22 22 10  0 22  8  6 15  6  1  6 15  6 15  6 15  6 15 10  0 22
  0 22  8 22  8  6 15  6 15  6 15  6 15 10  0 22  8 22 22 10  5  5  5  5
  1  6  1  5  5  5  5  1  6  1  5  5  5  1  6 15  6  1  5  5  1  6 15  6
  1  5  5  5  1  6  1  5  5  5  5  1  6  1  5  5  5  1  6 15  6  1  5  1
  6 15  6 15  6 15 10  0 22 22  8  6 15  6  1  5  1  6 15  6 15  6 15  6
 15  6 15 10  0 22  8 22  8]
Actu [0 1 1 1 1 1 1 1 6 6 6 6 6 6 6 1 1 8 8 8 8 1 1 1 1 8 8 8 8 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 6 6 6 6 6 6 6 6 1 1 8 8 8 8 1 1 1 1 8 8 8 8 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 8 8 8 8 8 8 8 8 8 8 8 8 8 8 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 8 8 8 8 8 8 8 1 1 8 8 8 8 1 1 1 1 8 8 8 8 8 8 8 1 1 1 1 1 1 8 8 8 8 1 1 1
 1 8 8 8 8]


In [19]:
for i in range(len(B)):
    if i in symbols:
        print(u"{}, {:0.4f}, {:0.4f}, {:0.4f}, {:0.4f}, {:0.4f}".format(i, *B[i,:]))

0, 0.0068, 0.0000, 0.0000, 0.0020, 0.0158
59, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
60, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
61, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
62, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
63, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
64, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
65, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
66, 0.0000, 0.0013, 0.0000, 0.0000, 0.0000
67, 0.0000, 0.0000, 0.0000, 0.0128, 0.0000
68, 0.0000, 0.0000, 0.0000, 0.0060, 0.0000
69, 0.0000, 0.0000, 0.0000, 0.0022, 0.0000
70, 0.0000, 0.0000, 0.0000, 0.1695, 0.0100
71, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
72, 0.0000, 0.0000, 0.0000, 0.0881, 0.0135
73, 0.0000, 0.0000, 0.0000, 0.0144, 0.0000
74, 0.0000, 0.0059, 0.0000, 0.3986, 0.0000
75, 0.0000, 0.0000, 0.0000, 0.2918, 0.3693
76, 0.0044, 0.0132, 0.0000, 0.0015, 0.0243
77, 0.0000, 0.0000, 0.0000, 0.0000, 0.5186
78, 0.0020, 0.0000, 0.0000, 0.0000, 0.0050
79, 0.0313, 0.0940, 0.0000, 0.0131, 0.0000
80, 0.0011, 0.0220, 0.0000, 0.0001, 0.0407
81, 0.0265, 